In [ ]:
#to know the directory with sputtern data (excel files)

%pwd

In [1]:
#to import various libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import matplotlib.ticker as plticker

#library for plotly
import plotly.express as px

from matplotlib.dates import DateFormatter
import matplotlib.ticker as plticker

In [ ]:
#to import a file in Jupyter Notebook

Sput_Number = input('Type a tool number (e.g. 5 (only a number!!) if you want data for Sput5): ')
df = pd.read_excel('Sput' + Sput_Number + '.xlsx', header=None) 
df.rename(columns={0:'abc'}, inplace=True)
df['abc'] = df['abc'].astype(str)

In [3]:
start = input('Type a desired start date. The following format applies: mm/dd/yyyy hh:mm. For example: 12/25/2022 17:31 ')
end = input('Type a desired end date. The following format applies: mm/dd/yyyy hh:mm. For example: 01/17/2023 13:55 ')

Type a desired start date. The following format applies: mm/dd/yyyy hh:mm. For example: 12/25/2022 17:31 06/25/2021 08:00
Type a desired end date. The following format applies: mm/dd/yyyy hh:mm. For example: 01/17/2023 13:55 06/25/2022 08:00


In [ ]:
#modification of imported data to select values between defined start and end dates

convert_s = pd.to_datetime(end, format="%m/%d/%Y %H:%M")
convert_e = pd.to_datetime(start, format="%m/%d/%Y %H:%M")
#-------------------------------------------------------------------------------------------
lis2 = []
lis4 = []
def func():        #function that gets you the position of rows (between the defined dates) - need that for func2()
    count1 = 0     #total count from the start of the excel file to the defined Timestamp 'end'
    count2 = 0     #count from the start of the excel file to the defined Timestamp 'start'
    for i in df['abc']:
        if i.startswith('END'):
            i2 = i[10:26]     #extract date and hours from the 'END DATE' line 
            i2 = pd.to_datetime(i2, format="%m/%d/%Y %H:%M")      #convert that to 'Datetime'
            count1 += 1         #start counting
            if convert_s >= i2:    
                lis2.append(i)        #append number of 'END DATE' to list 2
                if convert_e >= i2:
                    lis2.pop()        #remove the last element from the list 2 
                    count1 -= 1  
                    count3 = len(lis2)
                    count2 = count1 - count3
                    lis4.append(count2)        #you need these lines for the code in func2()
                    return lis4          #end the for loop   
#-------------------------------------------------------------------------------------------
lis3 = []
lis5 = []
def func2():      #function that generates list of rows from the excel file between the defined dates
    count4 = 0
    for i in df['abc']:
        lis3.append(i)
        if i.startswith('END'):
            count4 += 1
            i2 = i[10:26]
            i2 = pd.to_datetime(i2, format="%m/%d/%Y %H:%M")
            if convert_s >= i2:
                if count4 == (lis4[0] + 1):     #separate rows above the start date
                    lis5.append((lis3[-12:]))     #append recipes from the start date to a separate list
                    lis3.clear()                #discard rows above the start date
                    lis3.append(lis5[-1][-12])
                    lis3.append(lis5[-1][-11])
                    lis3.append(lis5[-1][-10])
                    lis3.append(lis5[-1][-9])
                    lis3.append(lis5[-1][-8])
                    lis3.append(lis5[-1][-7])
                    lis3.append(lis5[-1][-6])
                    lis3.append(lis5[-1][-5])
                    lis3.append(lis5[-1][-4])
                    lis3.append(lis5[-1][-3])
                    lis3.append(lis5[-1][-2])       #append recipes from start date to list3
                if convert_e >= i2:
                    lis3.pop()
                    del lis3[-3:]               #remove 2 recipes and an END DATE that you don't need
                    del lis3[-1]                #remove EQUIPMENT line that is not relevant
                    return ' '                  #stop the execution of the for loop


func()
func2()


In [5]:
df = pd.DataFrame()
df['abc'] = lis3

In [ ]:
#separation of rows that include equipment (with a chamber of interest), recipe and an APC violation

chamber = input('Type a chamber of interest (e.g. A (only a letter or a number!!)): ')

everything = []
def zusammen():
    for line in df['abc']:
        if line.startswith('EQUIPMENT: SPUT{}_{}'.format(Sput_Number, chamber)) or line.startswith('RECIPE') or \
        line.startswith('-9') or line.startswith('-8') or line.startswith('-7') or line.startswith('-6') or \
        line.startswith('-5') or line.startswith('-4') or line.startswith('-3') or line.startswith('-2') or \
        line.startswith('-1') or line.startswith('0') or line.startswith('1') or line.startswith('2') or \
        line.startswith('3') or line.startswith('4') or line.startswith('5') or line.startswith('6') or \
        line.startswith('7') or line.startswith('8') or line.startswith('9'):
            if line.startswith('-9') or line.startswith('-8') or line.startswith('-7') or line.startswith('-6') \
            or line.startswith('-5') or line.startswith('-4') or line.startswith('-3') or line.startswith('-2') \
            or line.startswith('-1') or line.startswith('0') or line.startswith('1') or line.startswith('2') or \
            line.startswith('3') or line.startswith('4') or line.startswith('5') or line.startswith('6') or \
            line.startswith('7') or line.startswith('8') or line.startswith('9'):
                a = line[127:]
                everything.append(a)
            else:
                everything.append(line)
                
zusammen()

In [7]:
#generator function  (separation of recipes and APC violations for a defined chamber)

everything2 = []

def zusammen2():
    buffer= []
    count = 0
#    lis2 = []
    for line2 in everything:
        if line2.startswith('EQUIPMENT: SPUT{}_{}'.format(Sput_Number, chamber)):
#            if buffer: 
#                yield None
            buffer = []   #None
            buffer = [line2]
            count = 1000000000
        else:
            buffer.append(line2)
            count+= 1
            if count == 1000000003:
                yield everything2.append(buffer)
#                return lis2
                count = 0
                buffer = []
#                yield lis2
#                buffer = []
    return None
#    return lis3.append(lis2)

In [8]:
#part of a generator function (separated in a new cell to avoid crashing)

generate = zusammen2()
while True:
    try:
        next(generate)
#        print(next(gen))
    except StopIteration:
        break
everything2

if len(everything2) == 0:
    print('\nNo APC violations appeared from {} to {} in Sput{}, Chamber {}.'.format(start, end, Sput_Number, chamber))

In [9]:
#removal of the 1st recipe from the list. (EPA recipe remains)

b1 = []
def iterator():
    for i in everything2:
        for b in i:
            b1.append(b)
    return b1

#iterator()
l1 = []
l2 = []
def iterator2():
    count = 0
    for a in b1:
        if a.startswith('RECIPE:') or a.startswith('RECIPEGROUP:') or a.startswith('RECIPENAME:'):
            if (count % 2) == 0 or count == 2:
                l2.append(a)
                count += 1
            else:
                l1.append(a)
                count += 1
        else:
            l1.append(a)

iterator()
iterator2()

In [ ]:
# graph illustrating number of APC violations that occured in a chamber vs a recipe 
# x-axis (recipes), y-axis (number of APC violations)

#for graphical illustration of results

d = {}
for c1 in l1:
    if c1.startswith('RECIPE:'):
        d[c1[8:]] = d.get(c1[8:], 0) + 1
        
e = sorted(d.items(), key=lambda x:x[1], reverse=True)   #sorted function returns a list
r = dict(e)

li = list()
for i in range(1, (len(r)+1)):
    li.append(i)

li2 = list()
for i2 in range(len(r)):
    li2.append(i2)
#-----------------------------------------------------------------------------------------------------------
#graphical illustration of APC violations for every recipe in the chamber    
    
fig, ax = plt.subplots()
ax.bar(r.keys(), r.values())
ax.set_xticks(li2)
ax.set_xticklabels(li)

ax.set_xlabel('Recipes in the tool')
ax.set_ylabel('Number of APC violations')

plt.show()

print('Report for Sput{}, Chamber {}.\n'.format(Sput_Number, chamber))
print('From {} to {}, the total number of APC violations was: {}.\n'.format(start, end, sum(r.values())))

for x,y,i in zip(r.keys(), r.values(), li):
    print('{} Represents recipe: "{}" that gave {} APC violations.'.format(i,x,y))

In [ ]:
#graph illustrating a number of APC violations in the defined chamber
# x-axis (different types of APC violations), y-axis (number of APC violations)

#to generate a list with APC violations

no1 = list()
ye1 = list()
for f1 in l1:
    if f1.startswith('EQUIPMENT') or f1.startswith('RECIPE:'):
        no1.append(f1)
    else:
        ye1.append(f1)

d2 = {}
for f2 in ye1:
    d2[f2] = d2.get(f2, 0) +1
        
e2 = sorted(d2.items(), key=lambda x:x[1], reverse=True)   #sorted function returns a list
r2 = dict(e2)
    
li3 = list()
for i in range(1, (len(r2)+1)):
    li3.append(i)
    
li4 = list()
for i2 in range(len(r2)):
    li4.append(i2)

#-----------------------------------------------------------------------------------------------------------    
#graphical illustration of the number of APC violations in the chamber

fig, ax = plt.subplots()
ax.bar(r2.keys(), r2.values())
#ax.set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
ax.set_xticks(li4)
ax.set_xticklabels(li3)

ax.set_xlabel('Various APC violations in the tool')
ax.set_ylabel('Number of APC violations')

plt.show()

print('Report for Sput{}, Chamber {}.\n'.format(Sput_Number, chamber))
print('From {} to {}, the total number of APC violations in the chamber was: {}.\n'.format(start, end, sum(r.values())))

for x,y,i in zip(r2.keys(), r2.values(), li3):
    print('{} Represents the following APC violation: "{}" whose count was {}.'.format(i,x,y))

In [12]:
del_list=[]
use_list=[]

for c3 in l1:
    if c3.startswith('EQUIPMENT:'):
        del_list.append(c3)
    else:
        use_list.append(c3)

list_recipes=[]
list_APC = []

for c5 in use_list:
    if c5.startswith('RECIPE'):
        list_recipes.append(c5)
    else:
        list_APC.append(c5)      


In [13]:
#https://stackoverflow.com/questions/5419204/index-of-duplicates-items-in-a-python-list

from collections import defaultdict

def list_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items())

source = list_recipes
lis7 = []
for dup in list_duplicates(source):
    lis7.append(dup)

list_recipes2 = []
for i in range(len(lis7)):
    list_recipes2.append(lis7[i][0])  

list_index = []
for i in range(len(lis7)):
    list_index.append(lis7[i][1])  

length_index = [0]    #start with 0 in the list

for i in list_index:
    length_index.append(len(i))
  

In [14]:
#sum values in length_index via increasing sequence
length_index_sum = []
for i8 in range(len(length_index)):
    b = sum(length_index[:i8+1])
    length_index_sum.append(b)  

In [15]:
list_recipes2   #in this list you have all recipes
list_index    #in this list you have index locations for every recipe above
length_index   #size of the indexes for every recipe
length_index_sum      #sum of length_index

list_location_APC = []
dict9 = {}


def abc():
    for i in range(len(list_recipes2)):
        list_index[i]
        for i2 in list_index[i]:
            list_location_APC.append(list_APC[i2])     #APC violations are saved based on the list_index
        
        dict9[list_recipes2[i]] = list_location_APC[length_index_sum[i]:length_index_sum[i+1]]
                 
abc()     

In [ ]:
recipe = input('Type (better to copy-paste from the 1st graph above) a recipe that you want to analyse: ') 

l4 = dict9['RECIPE: {}'.format(recipe)]

d4 = {}
for c4 in l4:
    d4[c4] = d4.get(c4, 0) + 1

In [ ]:
#graph illustrating a number of APC violations in the defined chamber for a defined recipe
# x-axis (different types of APC violations), y-axis (number of APC violations) for a defined recipe!!

e4 = sorted(d4.items(), key=lambda x:x[1], reverse=True)   #sorted function returns a list
r4 = dict(e4)

li4 = list()
for i4 in range(1, (len(r4)+1)):
    li4.append(i4)

li5 = list()
for i5 in range(len(r4)):
    li5.append(i5)

#-------------------------------------------------------------------------------------------------------------------    
fig, ax = plt.subplots()
ax.bar(r4.keys(), r4.values())
#ax.set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
ax.set_xticks(li5)
ax.set_xticklabels(li4)

ax.set_xlabel('Various APC violations in the tool')
ax.set_ylabel('Number of APC violations')

plt.show()

print('Report for RECIPE: {}.\n'.format(recipe))
print('From {} to {} -'.format(start, end) + ' Sput{}, Chamber {}.'.format(Sput_Number, chamber) \
      + ' The total number of APC violations was: {}.\n'.format(sum(r4.values())))
for x,y,i in zip(r4.keys(), r4.values(), li4):
#    print('{} Represents the following APC violation: "{}" that gave {} APC violations.'.format(i,x,y))
    print('{} Represents the following APC violation: "{}" whose count was {}.'.format(i,x,y))

In [ ]:
#all APC violations that occured in a tool (all chambers combined) during the selected timeframe

lis = []
dic = {}
count=0
for line in df['abc']:
    if line.startswith('-9') or line.startswith('-8') or line.startswith('-7') or line.startswith('-6') \
    or line.startswith('-5') or line.startswith('-4') or line.startswith('-3') or line.startswith('-2') \
    or line.startswith('-1') or line.startswith('0') or line.startswith('1') or line.startswith('2') \
    or line.startswith('3') or line.startswith('4') or line.startswith('5') or line.startswith('6') \
    or line.startswith('7') or line.startswith('8') or line.startswith('9'):
        count+=1   #to count the number of APC violations
        b=line[127:]    #to select the name of an APC violation
        dic[b]=dic.get(b,0) + 1     #to count the number of a specific APC violation

print('\n' + 'From {} to {}, there were total of {} APC violations in Sput{} (all chambers combined).'\
      .format(start, end, count, Sput_Number))

s = sorted(dic.items(), key=lambda x:x[1], reverse=True)   #to sort the dictionary by values, 1 represents values, keys are represented by 0
p = dict(s)
p

In [19]:
ls =[]
ls.append(lis2[0])

In [20]:
if len(lis2) == 1:
    count = 0
    count2 = 0
    for i in lis3:
        count2 += 1
        if i.startswith('REC'):
            count += 1
            if count == 2:
                lis3[count2:(count2+1)] = lis2
else:
    count = 0
    count2 = 0
    for i in lis3:
        count2 += 1
        if i.startswith('REC'):
            count += 1
            if count == 2:
                lis3[count2:(count2+1)] = ls
                break
    

In [21]:
df = pd.DataFrame()
df['abc'] = lis3

In [22]:
#separation of rows that include equipment (with a chamber of interest), an APC violation and an END DATE

everything = []
def zusammen():
    for line in df['abc']:
        if line.startswith('EQUIPMENT: SPUT{}_{}'.format(Sput_Number, chamber)) or line.startswith('RECIPE') or \
        line.startswith('-9') or line.startswith('-8') or line.startswith('-7') or line.startswith('-6') or \
        line.startswith('-5') or line.startswith('-4') or line.startswith('-3') or line.startswith('-2') or \
        line.startswith('-1') or line.startswith('0') or line.startswith('1') or line.startswith('2') or \
        line.startswith('3') or line.startswith('4') or line.startswith('5') or line.startswith('6') or \
        line.startswith('7') or line.startswith('8') or line.startswith('9') or line.startswith('END DATE'):
            if line.startswith('-9') or line.startswith('-8') or line.startswith('-7') or line.startswith('-6') \
            or line.startswith('-5') or line.startswith('-4') or line.startswith('-3') or line.startswith('-2') \
            or line.startswith('-1') or line.startswith('0') or line.startswith('1') or line.startswith('2') or \
            line.startswith('3') or line.startswith('4') or line.startswith('5') or line.startswith('6') or \
            line.startswith('7') or line.startswith('8') or line.startswith('9'):
                a = line[127:]
                everything.append(a)
            else:
                everything.append(line)
                
zusammen()

In [23]:
#generator function 

everything2 = []

def zusammen2():
    buffer= []
    count = 0
#    lis2 = []
    for line2 in everything:
        if line2.startswith('EQUIPMENT: SPUT{}_{}'.format(Sput_Number, chamber)):
#            if buffer: 
#                yield None
            buffer = []   #None
            buffer = [line2]
            count = 1000000000
        else:
            buffer.append(line2)
            count+= 1
            if count == 1000000003:
                yield everything2.append(buffer)
#                return lis2
                count = 0
                buffer = []
#                yield lis2
#                buffer = []
    return None
#    return lis3.append(lis2)

In [24]:
#part of a generator function (separated in a new cell to avoid crashing)

generate = zusammen2()
while True:
    try:
        next(generate)
#        print(next(gen))
    except StopIteration:
        break
everything2

if len(everything2) == 0:
    print('\nNo APC violations appeared from {} to {} in Sput{}, Chamber {}.'.format(start, end, Sput_Number, chamber))

In [25]:
#removal of the 1st recipe from the list. (EPA recipe remains)

b1 = []
def iterator():
    for i in everything2:
        for b in i:
            b1.append(b)
    return b1

#iterator()
l1 = []
l2 = []
def iterator2():
    count = 0
    for a in b1:
        if a.startswith('RECIPE:') or a.startswith('RECIPEGROUP:') or a.startswith('RECIPENAME:'):
            if (count % 2) == 0 or count == 2:
                l2.append(a)
                count += 1
            else:
                l1.append(a)
                count += 1
        else:
            l1.append(a)

iterator()
iterator2()

In [26]:
del_list=[]
use_list=[]

for c3 in l1:
    if c3.startswith('EQUIPMENT:'):
        del_list.append(c3)
    else:
        use_list.append(c3)

list_recipes=[]
list_APC = []

for c5 in use_list:
    if c5.startswith('RECIPE'):
        list_recipes.append(c5)
    else:
        list_APC.append(c5)      


In [27]:
#https://stackoverflow.com/questions/5419204/index-of-duplicates-items-in-a-python-list

from collections import defaultdict

def list_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items())

source = list_recipes
lis7 = []
for dup in list_duplicates(source):
    lis7.append(dup)

list_recipes2 = []
for i in range(len(lis7)):
    list_recipes2.append(lis7[i][0])  

list_index = []
for i in range(len(lis7)):
    list_index.append(lis7[i][1])  

length_index = [0]    #start with 0 in the list

for i in list_index:
    length_index.append(len(i))
  

In [28]:
#sum values in length_index via increasing sequence
length_index_sum = []
for i8 in range(len(length_index)):
    b = sum(length_index[:i8+1])
    length_index_sum.append(b)  

In [29]:
list_recipes2   #in this list you have all recipes
list_index    #in this list you have index locations for every recipe above
length_index   #size of the indexes for every recipe
length_index_sum      #sum of length_index

list_location_APC = []
dict9 = {}


def abc():
    for i in range(len(list_recipes2)):
        list_index[i]
        for i2 in list_index[i]:
            list_location_APC.append(list_APC[i2])     #APC violations are saved based on the list_index
        
        dict9[list_recipes2[i]] = list_location_APC[length_index_sum[i]:length_index_sum[i+1]]
                 
abc()     

In [30]:
#recipe = input('Type (better to copy-paste from the 1st graph above) a recipe that you want to analyse (e.g. 6NIAL2.6 40): ') 

l4 = dict9['RECIPE: {}'.format(recipe)]

d4 = {}
for c4 in l4:
    d4[c4] = d4.get(c4, 0) + 1

In [31]:
e4 = sorted(d4.items(), key=lambda x:x[1], reverse=True)   #sorted function returns a list
r4 = dict(e4)

In [ ]:
#graphical illustration of the number of APC violations during the selected timeframe (for a defined recipe in a defined ch)
# x-axis (timeframe), y-axis (number of APC violations)

dict_data5 = {}
for key, value in r4.items():
    dict_data5[key[10:26]] = value
    
dict_data6 = {}
for key, value in dict_data5.items():
    key = pd.to_datetime(key, format="%m/%d/%Y %H:%M")
    dict_data6[key] = value

a = pd.date_range(convert_e , convert_s, freq='T')    #generate data between defined range

#-----------------------------------------------------------------------------------------------------------    
# for graphical illustration
#if your Timestamp is between a time period of a few minutes, remove: date_form, ax.xaxis.set, loc and ax.xaxis.set 

fig, ax = plt.subplots()
ax.scatter(dict_data6.keys(), dict_data6.values())
ax.set_xlim(left=a[0], right=a[-1])

# Change based on the selected date range    https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
date_form = DateFormatter('%y-%m')       # for years add %y, for months %m  - %m-%d
ax.xaxis.set_major_formatter(date_form)
# Change based on the selected date range
ax.set_xlabel('Timestamp (yy-mm)')      #for years add yy, for months mm - mm-dd

ax.set_ylabel('Number of APC violations')

# Change based on the selected date range
loc = plticker.MultipleLocator(base=60) 
ax.xaxis.set_major_locator(loc)

plt.yticks(np.arange(min(dict_data5.values()), max(dict_data5.values())+1, 1.0))

plt.show()

print('Report for Sput{}, Chamber {}, Recipe: {} (from {} to {}).'.format(Sput_Number, chamber, recipe, start, end))

In [ ]:
#graphical illustration of the number of APC violations during the selected timeframe (only for a defined chamber)
# x-axis (timeframe), y-axis (number of APC violations)

date_list = []
for i in everything2:
    for i2 in i:
        date_list.append(i2)

dict_data3 = {}
for i in date_list:
    if i.startswith('END DATE'):
        dict_data3[i[10:26]] = dict_data3.get(i[10:26], 0) + 1


dict_data4 = {}
for key, value in dict_data3.items():
    key = pd.to_datetime(key, format="%m/%d/%Y %H:%M")
    dict_data4[key] = value

# create a DataFrame
#data2 = pd.DataFrame()
#data2['Keys'] = dict_data2.keys()
#data2['Values'] = dict_data2.values()

a = pd.date_range(convert_e , convert_s, freq='T')    #generate data between defined range

#-----------------------------------------------------------------------------------------------------------    
# for graphical illustration
#if your Timestamp is between a time period of a few minutes, remove: date_form, ax.xaxis.set, loc and ax.xaxis.set 

fig, ax = plt.subplots()
ax.scatter(dict_data4.keys(), dict_data4.values())
ax.set_xlim(left=a[0], right=a[-1])

# Change based on the selected date range
date_form = DateFormatter('%y-%m')       # for years add %y
ax.xaxis.set_major_formatter(date_form)
# Change based on the selected date range
ax.set_xlabel('Timestamp (yy-mm)')      #for years add yy

ax.set_ylabel('Number of APC violations')

# Change based on the selected date range
loc = plticker.MultipleLocator(base=50) 
ax.xaxis.set_major_locator(loc)

plt.yticks(np.arange(min(dict_data4.values()), max(dict_data4.values())+1, 1.0))

plt.show()

print('Report for Sput{}, Chamber {} (from {} to {}).'.format(Sput_Number, chamber, start, end))

In [ ]:
#graphical illustration of the number of APC violations during the selected timeframe (all chambers combined)
# x-axis (timeframe), y-axis (number of APC violations)

dict_data = {}
for i in lis2:
    dict_data[i[10:26]] = dict_data.get(i[10:26], 0) + 1

dict_data2 = {}
for key, value in dict_data.items():
    key = pd.to_datetime(key, format="%m/%d/%Y %H:%M")
    dict_data2[key] = value

# create a DataFrame
#data2 = pd.DataFrame()
#data2['Keys'] = dict_data2.keys()
#data2['Values'] = dict_data2.values()

a = pd.date_range(convert_e , convert_s, freq='T')    #generate data between defined range

#-----------------------------------------------------------------------------------------------------------    
# for graphical illustration

fig, ax = plt.subplots()
ax.scatter(dict_data2.keys(), dict_data2.values())
ax.set_xlim(left=a[0], right=a[-1])

# Change based on the selected date range
date_form = DateFormatter('%y-%m-%d')       # for years add %y
ax.xaxis.set_major_formatter(date_form)
# Change based on the selected date range
ax.set_xlabel('Timestamp (mm-dd)')      #for years add yy

ax.set_ylabel('Number of APC violations')

# Change based on the selected date range
loc = plticker.MultipleLocator(base=60) 
ax.xaxis.set_major_locator(loc)

plt.yticks(np.arange(min(dict_data2.values()), max(dict_data2.values())+1, 1.0))

plt.show()

print('Report for Sput{} (from {} to {}) - all chambers combined.'.format(Sput_Number, start, end))
